In [2]:

import pandas as pd
from tqdm import tqdm
from sentence_transformers import InputExample, models
from torch import nn
import torch.optim as optim
import torch


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Using an already pretrained SBERT to compute cosine similarities which would then be used 

df=pd.read_csv("stsb_train.csv")
df=df.drop("idx",axis=1)
df_cosine_labels = df.copy() 
df_cosine_labels["label"] = df_cosine_labels["label"] /df_cosine_labels["label"].abs().max() 


df=pd.read_csv("stsb_validation.csv")
df=df.drop("idx",axis=1)
validation_cosine_labels = df.copy() 
validation_cosine_labels["label"] = validation_cosine_labels["label"] /validation_cosine_labels["label"].abs().max() 



In [4]:

from torch.utils.data import DataLoader
train_examples = []
for i,row in df_cosine_labels.iterrows():
    train_examples.append([[row["sentence1"], row["sentence2"]],row["label"]])

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

val_examples = []
for i,row in validation_cosine_labels.iterrows():
    val_examples.append([[row["sentence1"], row["sentence2"]],row["label"]])

val_dataloader = DataLoader(val_examples, shuffle=True, batch_size=16)

In [17]:
from transformers import DistilBertModel,DistilBertTokenizer
class ShroomBasHoGayaBhai(nn.Module):
    def __init__(self,max_seq_length):
        super(ShroomBasHoGayaBhai, self).__init__()
        self.word_embedding_model = models.Transformer("distilbert-base-uncased", max_seq_length=max_seq_length)
        self.pooling_model = models.Pooling(self.word_embedding_model.get_word_embedding_dimension())
        self.dense_model = models.Dense(
            in_features=self.pooling_model.get_sentence_embedding_dimension(),
            out_features=256,
            activation_function=nn.Tanh(),
        )
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)

    def forward_once(self, x):
        embedding = self.word_embedding_model(x)
        embedding = self.pooling_model(embedding)
        embedding = self.dense_model(embedding)
 

        return embedding

    def forward(self, sentence1, sentence2):
        output1 = self.forward_once(sentence1)["sentence_embedding"]
        output2 = self.forward_once(sentence2)["sentence_embedding"]
        similarity_score = self.cos(output1, output2)
  
        return similarity_score

def encode_pair(pair,padding=True,truncation=True,max_length=256):
    tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
    return tokenizer(
    pair[0], padding=padding,truncation=truncation,max_length=max_length, return_tensors="pt"
    ), tokenizer(
        pair[1],padding=padding,truncation=truncation,max_length=max_length, return_tensors="pt"
    )

model = ShroomBasHoGayaBhai(256)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0000001)
num_epochs=50
best_val_loss = float('inf')
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total parameters in the model:", total_params)
for epoch in tqdm(range(num_epochs)):
    running_loss = 0.0
    correct = 0
    total = 0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        if i%100==0:
            print(f"Batch: {i+1}")
        sentences, labels = data
        optimizer.zero_grad()
        sentence1,sentence2=encode_pair(sentences)
        labels = labels.float()
        outputs=model(sentence1,sentence2)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_data in val_dataloader:
            val_sentences, val_labels = val_data
            val_sentence1, val_sentence2 = encode_pair(val_sentences)
            val_labels = val_labels.float()
            val_outputs = model(val_sentence1, val_sentence2)
            val_loss += criterion(val_outputs, val_labels).item()
    val_loss /= len(val_dataloader)

    print(f'Epoch {epoch+1} | Train Loss: {running_loss / len(train_dataloader)} | Validation Loss: {val_loss}')

    # if val_loss < best_val_loss:
    #     torch.save(model.state_dict(), "best_model_v2_lr1.pth")
    #     best_val_loss = val_loss


Total parameters in the model: 66559744


  0%|          | 0/50 [00:00<?, ?it/s]

Batch: 1
Batch: 101
Batch: 201
Batch: 301


  2%|▏         | 1/50 [03:59<3:15:38, 239.56s/it]

Epoch 1 | Train Loss: 0.17361739428920878 | Validation Loss: 0.18584504137013821
Batch: 1
Batch: 101
Batch: 201
Batch: 301


...

Epoch 48 | Train Loss: 0.027106327426412866 | Validation Loss: 0.0320096748405473
Batch: 1
Batch: 101
Batch: 201
Batch: 301


 98%|█████████▊| 49/50 [3:17:43<03:59, 239.61s/it]

Epoch 49 | Train Loss: 0.02661641671632727 | Validation Loss: 0.03185276660394478
Batch: 1
Batch: 101
Batch: 201
Batch: 301


100%|██████████| 50/50 [3:21:52<00:00, 242.25s/it]

Epoch 50 | Train Loss: 0.026571826328937377 | Validation Loss: 0.03165640120216189


NameError: name 'encode_pair' is not defined

In [9]:

state_dict = torch.load("best_model_v2_lr1.pth")
model= ShroomBasHoGayaBhai(256)


model.load_state_dict(state_dict)



OrderedDict([('word_embedding_model.auto_model.embeddings.word_embeddings.weight', tensor([[-0.0166, -0.0666, -0.0163,  ..., -0.0200, -0.0514, -0.0264],
        [-0.0132, -0.0673, -0.0161,  ..., -0.0227, -0.0554, -0.0260],
        [-0.0176, -0.0709, -0.0144,  ..., -0.0246, -0.0596, -0.0232],
        ...,
        [-0.0231, -0.0588, -0.0105,  ..., -0.0195, -0.0262, -0.0212],
        [-0.0490, -0.0561, -0.0047,  ..., -0.0107, -0.0180, -0.0219],
        [-0.0065, -0.0915, -0.0025,  ..., -0.0151, -0.0504,  0.0460]])), ('word_embedding_model.auto_model.embeddings.position_embeddings.weight', tensor([[ 0.0180, -0.0245, -0.0364,  ...,  0.0004,  0.0003,  0.0153],
        [ 0.0079,  0.0021, -0.0181,  ...,  0.0294,  0.0300, -0.0047],
        [-0.0112, -0.0019, -0.0111,  ...,  0.0164,  0.0189, -0.0080],
        ...,
        [ 0.0174,  0.0035, -0.0096,  ...,  0.0030,  0.0004, -0.0269],
        [ 0.0217, -0.0060,  0.0147,  ..., -0.0056, -0.0126, -0.0281],
        [ 0.0026, -0.0233,  0.0055,  ...,  0

<All keys matched successfully>

In [14]:
sentence1,sentence2=encode_pair([
"A person who cobbles .","Nonsense ."
])
outputs=model(sentence1,sentence2)
print(outputs)

tensor([0.2301], grad_fn=<SumBackward1>)
